# How to setup Coordle index and use it

In [1]:
import pandas as pd 
from coordle_backend import (CordDoc, Index, RecursiveDescentParser, 
                             AI_Index, DocWordCounts, WordToDoc)

from gensim.models import Word2Vec
from os.path import join as join_path
from gensim.models.callbacks import CallbackAny2Vec

Gotta have this shit

In [2]:
class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''
class DocEpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

In [3]:
# Load the last trained model
model = Word2Vec.load(join_path('models-word2vec-new', 'model_epoch_20.model'))

Load pd.DataFrame, note that only loading first 8196 rows to save time here

In [4]:
df = pd.read_csv('data/cord-19-data.csv', nrows=8196)

Instantiate AI_Index and index df 

In [5]:
ai_index = AI_Index(model.wv.most_similar, n_similars=1)
ai_index.build_from_df(
    df=df,
    uid='cord_uid',
    title='title',
    text='body_text', 
    verbose=True, 
    use_multiprocessing=True,
    workers=-1
)

Text cleaning initilized on 16 workers


Adding to index: 100%|██████████| 8196/8196 [00:25<00:00, 320.57it/s]


Searching with a proper query

In [6]:
docs, scores, errmsgs = ai_index.search('white AND retarded woman')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}  {str(doc.title)[:70]:<70}  {score:.4f}')

g42p98ku  Gliopathy of Demyelinating and Non-Demyelinating Strains of Mouse Hepa  4.0658
c33lqwua  Risky Bodies in the Plasma Bioeconomy: A Feminist Analysis              3.7543
sbixelq5  Proteomic Profiling of the Amniotic Fluid to Detect Inflammation, Infe  2.0642
g8m6n1u3  A DESCRIPTIVE STUDY OF PANDEMIC INFLUENZA A(H1N1)PDM09 IN BRAZIL, 2009  1.8357
thkzoirr  A Flood of Health Functional Foods: What Is to Be Recommended?          1.4301
i3ydg93k  Biomarkers for Chronic Kidney Disease Associated with High Salt Intake  1.3663
i5p3vbfr  Worry experienced during the 2015 Middle East Respiratory Syndrome (ME  1.2909
d28li5ao  Congenital Malaria in China                                             1.2860
y65lguub  Migration and health in Southern Africa: 100 years and still circulati  1.2481
cd8u42xo  A Case of Acute Disseminated Encephalomyelitis Associated with Hepatit  1.1357
zgz4snfu  MicroRNA-155 enhances T cell trafficking and antiviral effector functi  1.1099
19jo817j  High Pulmon

Searching with bad query

In [7]:
docs, scores, errmsgs = ai_index.search('AND (white AND retarded woman) OR OR (')

if errmsgs:
    print(errmsgs)
else:
    for doc, score in zip(docs, scores):
        print(f'{doc.uid}  {str(doc.title)[:70]:<70}  {score:.4f}')

['SyntaxError: First token "AND" is an operator', 'SyntaxError: Two succeeding operators "OR OR"', 'SyntaxError: Found stray opening parenthesis']
